# Pythran

- [Documentation](https://pythonhosted.org/pythran/)
- [GitHub](https://github.com/serge-sans-paille/pythran)
- [Pythran tutorial](http://serge-sans-paille.github.io/pythran-stories/pythran-tutorial.html)
- [Mailing list](http://www.freelists.org/list/pythran)
- [StackOverflow](http://stackoverflow.com/questions/tagged/pythran)

"Pythran is a Python to c++ compiler for a subset of the Python language, with a focus on scientific computing. It takes a Python module annotated with a few interface description and turns it into a native python module with the same interface, but (hopefully) faster."

```bash
brew install gcc gmp openblas python3
pip3 install pythran
```

Skip the cell below if you are not on mac

In [1]:
%%file ~/.pythranrc 
[compiler]
include_dirs=/usr/local/opt/openblas/include
library_dirs=/usr/local/opt/openblas/lib
blas=openblas
CXX=g++-7
CC=gcc-7


Overwriting /Users/navaro/.pythranrc


# Introduction with Pi computation

Computing $\pi$ with a Fortran-like version:

In [2]:
def pi_approximate(n):
    step = 1.0 / n
    result = 0   
    for i in range(n):
        x = (i + 0.5) * step
        result += 4.0 / (1.0 + x * x)
    return step * result

pi_approximate(1000000)

3.1415926535897643

In [3]:
%timeit pi_approximate(1000000)

175 ms ± 3.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
import pythran

WARNING  Pythran support disabled for module: omp


In [5]:
%load_ext pythran.magic

The additionnal line beginning by #pythran give information about the argument type. The return type is infered.

In [6]:
%%file pi_approximate_pythran.py
#pythran export pi_approximate_pythran(int)
import numpy as np
def pi_approximate_pythran(n):
    step = 1.0 / n
    result = 0   
    for i in range(n):
        x = (i + 0.5) * step
        result += 4.0 / (1.0 + x * x)
    return step * result

Writing pi_approximate_pythran.py


In [7]:
!pythran -e pi_approximate_pythran.py
!cat pi_approximate_pythran.cpp

WARNING  Pythran support disabled for module: omp
#undef ENABLE_PYTHON_MODULE
#define BOOST_SIMD_NO_STRICT_ALIASING 1
#include <pythonic/core.hpp>
#include <pythonic/python/core.hpp>
#include <pythonic/types/bool.hpp>
#include <pythonic/types/int.hpp>
#ifdef _OPENMP
#include <omp.h>
#endif
#include <pythonic/include/types/int.hpp>
#include <pythonic/types/int.hpp>
#include <pythonic/include/numpy/square.hpp>
#include <pythonic/include/operator_/div.hpp>
#include <pythonic/include/__builtin__/range.hpp>
#include <pythonic/numpy/square.hpp>
#include <pythonic/operator_/div.hpp>
#include <pythonic/__builtin__/range.hpp>
namespace __pythran_pi_approximate_pythran
{
  struct pi_approximate_pythran
  {
    typedef void callable;
    typedef void pure;
    template <typename argument_type0 >
    struct type
    {
      typedef double __type0;
      typedef typename std::remove_cv<typename std::remove_reference<argument_type0>::type>::type __type1;
      typedef typename pythonic::assignable<d

Hopefully, the code behaves the same:

In [8]:
%%pythran
#pythran export pi_approximate_pythran(int)
import numpy as np
def pi_approximate_pythran(n):
    step = 1.0 / n
    result = 0   
    for i in range(n):
        x = (i + 0.5) * step
        result += 4.0 / (1.0 + x * x)
    return step * result

In file included from /usr/local/lib/python3.6/site-packages/pythran/pythonic/__builtin__/range.hpp:6:0,
                 from /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmpmq346ndz.cpp:14:
/usr/local/lib/python3.6/site-packages/pythran/pythonic/__builtin__/xrange.hpp:102:39: warning: 'pythonic::__builtin__::xrange::reverse_iterator pythonic::__builtin__::xrange::rend() const' defined but not used [-Wunused-function]
     typename xrange::reverse_iterator xrange::rend() const
                                       ^~~~~~
/usr/local/lib/python3.6/site-packages/pythran/pythonic/__builtin__/xrange.hpp:97:39: warning: 'pythonic::__builtin__::xrange::reverse_iterator pythonic::__builtin__::xrange::rbegin() const' defined but not used [-Wunused-function]
     typename xrange::reverse_iterator xrange::rbegin() const
                                       ^~~~~~
/usr/local/lib/python3.6/site-packages/pythran/pythonic/__builtin__/xrange.hpp:68:10: warning: 'bool pythonic::__builtin__::{an

In [9]:
pi_approximate_pythran(1000000)

3.1415926535897643

In [10]:
%timeit pi_approximate_pythran(1000000)

3.63 ms ± 5.31 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The loop can run in parallel with OpenMP integration:

In [11]:
%%pythran -fopenmp
#pythran export pi_approximate_pythran_omp(int)
import numpy as np
def pi_approximate_pythran_omp(n):
    step = 1.0 / n
    result = 0
    #omp parallel for reduction(+:result)
    for i in range(n):
        x = (i + 0.5) * step
        result += 4.0 / (1.0 + x * x)
    return step * result

In file included from /usr/local/lib/python3.6/site-packages/pythran/pythonic/__builtin__/range.hpp:6:0,
                 from /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmpzfepni2i.cpp:14:
/usr/local/lib/python3.6/site-packages/pythran/pythonic/__builtin__/xrange.hpp:102:39: warning: 'pythonic::__builtin__::xrange::reverse_iterator pythonic::__builtin__::xrange::rend() const' defined but not used [-Wunused-function]
     typename xrange::reverse_iterator xrange::rend() const
                                       ^~~~~~
/usr/local/lib/python3.6/site-packages/pythran/pythonic/__builtin__/xrange.hpp:97:39: warning: 'pythonic::__builtin__::xrange::reverse_iterator pythonic::__builtin__::xrange::rbegin() const' defined but not used [-Wunused-function]
     typename xrange::reverse_iterator xrange::rbegin() const
                                       ^~~~~~
/usr/local/lib/python3.6/site-packages/pythran/pythonic/__builtin__/xrange.hpp:68:10: warning: 'bool pythonic::__builtin__::{an

In [12]:
%timeit pi_approximate_pythran_omp(1000000)

979 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Vectorized code

In [13]:
import numpy as np
def pi_approximation_numpy(n):
    step = 1.0 / n
    x = (np.arange(0, n, dtype=np.float64) + 0.5) * step
    return step * np.sum(4. / (1. + x ** 2))

In [14]:
pi_approximation_numpy(1000000)

3.1415926535898775

In [15]:
%timeit pi_approximation_numpy(1000000)

9.15 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The cell below does not work with Python3 (Pythran Version 0.8.2)

 - Pythran supports the vectorized code only for Python 2
 - SIMD code generation (-DUSE_BOOST_SIMD -march=native)

In [17]:
%%pythran
#pythran export pi_numpy_style_pythran(int)
import numpy as np
def pi_numpy_style_pythran(n):
    step = 1.0 / n
    x = (np.arange(0, n, dtype=np.float64) + 0.5) * step
    return step * np.sum(4. / (1. + x ** 2))


In file included from /usr/local/lib/python3.6/site-packages/pythran/pythonic/types/ndarray.hpp:37:0,
                 from /usr/local/lib/python3.6/site-packages/pythran/pythonic/numpy/arange.hpp:7,
                 from /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmpbddbhjpc.cpp:17:
/usr/local/lib/python3.6/site-packages/pythran/pythonic/types/numpy_fexpr.hpp:203:10: error: prototype for 'decltype (((const pythonic::types::numpy_fexpr<A, F>*)this)->pythonic::types::numpy_fexpr<A, F>::fast(i)) pythonic::types::numpy_fexpr<A, F>::operator[](long int) const' does not match any in class 'pythonic::types::numpy_fexpr<A, F>'
     auto numpy_fexpr<Arg, F>::operator[](long i) const
          ^~~~~~~~~~~~~~~~~~~
In file included from /usr/local/lib/python3.6/site-packages/pythran/pythonic/include/types/ndarray.hpp:35:0,
                 from /usr/local/lib/python3.6/site-packages/pythran/pythonic/include/numpy/arange.hpp:5,
                 from /var/folders/mw/nwq5qyg56fl585pb9jw81_qc00

/usr/local/lib/python3.6/site-packages/pythran/pythonic/types/numpy_expr.hpp:22:27: warning: comparison between signed and unsigned integer expressions [-Wsign-compare]
In file included from /usr/local/lib/python3.6/site-packages/pythran/pythonic/types/ndarray.hpp:37:0,
                 from /usr/local/lib/python3.6/site-packages/pythran/pythonic/numpy/arange.hpp:7,
                 from /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmpbddbhjpc.cpp:17:
/usr/local/lib/python3.6/site-packages/pythran/pythonic/types/numpy_fexpr.hpp:203:10: error: prototype for 'decltype (((const pythonic::types::numpy_fexpr<A, F>*)this)->pythonic::types::numpy_fexpr<A, F>::fast(i)) pythonic::types::numpy_fexpr<A, F>::operator[](long int) const' does not match any in class 'pythonic::types::numpy_fexpr<A, F>'
     auto numpy_fexpr<Arg, F>::operator[](long i) const
          ^~~~~~~~~~~~~~~~~~~
In file included from /usr/local/lib/python3.6/site-packages/pythran/pythonic/include/types/ndarray.hpp:35:0,
 

/usr/local/lib/python3.6/site-packages/pythran/pythonic/types/numpy_expr.hpp:22:27: warning: comparison between signed and unsigned integer expressions [-Wsign-compare]


WARNING  Compilation error, trying hard to find its origin...
WARNING  Nop, I'm going to flood you with C++ errors!


CompileError: error: Command "g++-7 -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -DUSE_GMP -DBOOST_SIMD_CONFIG_SUPPORT_POSIX_MEMALIGN -DENABLE_PYTHON_MODULE -D__PYTHRAN__=3 -I/usr/local/opt/openblas/include -I/usr/local/lib/python3.6/site-packages/pythran -I/usr/local/lib/python3.6/site-packages/numpy/core/include -I/usr/local/lib/python3.6/site-packages/numpy/core/include -I/usr/local/include -I/usr/local/opt/openssl/include -I/usr/local/opt/sqlite/include -I/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/include/python3.6m -c /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmpbddbhjpc.cpp -o /var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmp6gdd_ne_/var/folders/mw/nwq5qyg56fl585pb9jw81_qc0000gn/T/tmpbddbhjpc.o -std=c++11 -fno-math-errno" failed with exit status 1

# Type Annotations

Consider the following function:

In [145]:
%%pythran
#pythran export f_pythran(int, int)
def f_pythran(u, v):
    return u+v

In [146]:
f_pythran(1.0,2.0)

TypeError: Invalid argument type for pythranized function `f_pythran'.
Candidates are:
   f_pythran(long,long)


Use double export to specify the overloads:

In [148]:
%%pythran
#pythran export f_pythran(int, int)
#pythran export f_pythran(float64, float64)
def f_pythran(u, v):
    return u+v

In [151]:
f_pythran(1.0,2.0), f_pythran(1,1)

(3.0, 2)

# Using Pythran from the Command Line

1. Write your code to Pythranize into a seperate file;
2. Call the Pythran compiler.

In [152]:
%%file scrabble.py
#pythran export scrabble_score(str, str:int dict)
def scrabble_score(word, scoretable):
    score = 0
    for letter in word:
        if letter in scoretable:
            score += scoretable[letter]
    return score


Overwriting scrabble.py


In [153]:
!python3 -m pythran.run -v scrabble.py

WARNING  Pythran support disabled for module: omp
running build_ext
running build_src
build_src
building extension "scrabble" sources
build_src: building npy-pkg config files
new_compiler returns <class 'distutils.unixccompiler.UnixCCompiler'>
INFO     customize UnixCCompiler
customize UnixCCompiler using build_ext
********************************************************************************
<class 'distutils.unixccompiler.UnixCCompiler'>
preprocessor  = ['gcc-7', '-E']
compiler      = ['gcc-7', '-Wno-unused-result', '-Wsign-compare', '-Wunreachable-code', '-fno-common', '-dynamic', '-DNDEBUG', '-g', '-fwrapv', '-O3', '-Wall', '-Wstrict-prototypes']
compiler_so   = ['gcc-7', '-Wno-unused-result', '-Wsign-compare', '-Wunreachable-code', '-fno-common', '-dynamic', '-DNDEBUG', '-g', '-fwrapv', '-O3', '-Wall', '-Wstrict-prototypes']
compiler_cxx  = ['g++-7']
linker_so     = ['gcc-7', '-bundle', '-undefined', 'dynamic_lookup']
linker_exe    = ['gcc-7']
archiver      = ['ar', 'rc']
ranlib

In [1]:
import scrabble

In [2]:
scrabble.__file__

'/Users/navaro/notebooks-python/scrabble.so'

In [3]:
scrabble.scrabble_score("hello", {"h": 4, "e": 1, "l": 1, "o": 1})

8

# Pyhran setup.py

```python
from distutils.core import Extension
from setuptools import setup, dist

dist.Distribution(dict(setup_requires='pythran'))

from pythran import PythranExtension
module1 = PythranExtension('demo', sources = ['scrabble.py'])

setup(name = 'demo',
      version = '1.0',
      description = 'This is a demo package',
      ext_modules = [module1])
```

## Functions as regular values

In [8]:
%%pythran
#pythran export modify(int, str)
actions = {"increase": lambda x: x + 1,
           "decrease": lambda x: x - 1}

def modify(value, action):
    what = actions[action]
    return what(value)

In [9]:
modify(1, "increase")

2

Passing functions in and out is not supported.